In [23]:
import sys
import os

%load_ext autoreload
%autoreload 2

import zarr
import numpy as np
import torch
import copy
import numcodecs
import shortuuid

from pathlib import Path

import gym_pusht  # noqa: F401
import gymnasium as gym
import imageio
import numpy
import torch
from huggingface_hub import snapshot_download

from lerobot.common.policies.diffusion.modeling_diffusion import DiffusionPolicy

from lerobot.common.datasets.push_dataset_to_hub.utils import (
    concatenate_episodes,
    get_default_encoding,
    save_images_concurrently,
)
from lerobot.common.datasets.utils import (
    calculate_episode_data_index,
    hf_transform_to_torch,
)
from lerobot.common.datasets.lerobot_dataset import CODEBASE_VERSION
from IPython.display import Video
from lerobot.common.datasets.rollout_datasets.episode_stores import EpisodeVideoStore

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
pretrained_policy_path = Path(snapshot_download("lerobot/diffusion_pusht"))

policy = DiffusionPolicy.from_pretrained(pretrained_policy_path)
policy.eval()

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available. Device set to:", device)
else:
    device = torch.device("cpu")
    print(f"GPU is not available. Device set to: {device}. Inference will be slower than on GPU.")
    # Decrease the number of reverse-diffusion steps (trades off a bit of quality for 10x speed)
    policy.diffusion.num_inference_steps = 10

policy = policy.to(device)

Fetching 11 files: 100%|█████████████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 220752.84it/s]


Loading weights from local directory
GPU is available. Device set to: cuda


In [25]:
output_directory = Path("outputs/eval/episode_store_test")
output_directory.mkdir(parents=True, exist_ok=True)

videos_dir = output_directory / 'videos'
videos_dir.mkdir(parents=True, exist_ok=True)
fps = 10
info = {
    "codebase_version": CODEBASE_VERSION,
    "fps": fps,
    "video": True,
    "encoding": get_default_encoding(),
    "videos_dir": str(videos_dir)
}           

In [26]:
env = gym.make(
    "gym_pusht/PushT-v0",
    obs_type="pixels_agent_pos",
    max_episode_steps=300,
)

In [27]:
episode_video_store = EpisodeVideoStore.create_from_path(output_directory, info, mode='a')
print(episode_video_store.num_episodes)
print(episode_video_store.info)

Connected to the exising zarr
5
{'codebase_version': 'v1.6', 'fps': 10, 'video': True, 'encoding': {'vcodec': 'libsvtav1', 'pix_fmt': 'yuv420p', 'g': 2, 'crf': 30}, 'videos_dir': 'outputs/eval/episode_store_test/videos'}


In [28]:
def build_ep_dict(observation_states: list, 
                  actions: list, 
                  rewards: list, 
                  dones: list, 
                  successes: list, 
                  frames: list, 
                  videos_dir: Path,
                  fps=10):
    num_frames = len(frames)
    assert len(observation_states) == num_frames
    assert len(actions) == num_frames
    assert len(rewards) == num_frames
    assert len(dones) == num_frames
    assert len(successes) == num_frames
    
    ep_dict = {}
    
    #  observation.image
    img_key = 'observation.image'
    fname = f"{img_key}_episode_{shortuuid.ShortUUID().random(length=8)}.mp4"
    video_path = videos_dir / fname
    imageio.mimsave(video_path, frames, fps=fps)
    print(f"Video of the evaluation is available in '{video_path}'.")
    ep_dict[img_key] = [
        {'path': f"videos/{fname}", 'timestamp': i / fps} for i in range(num_frames)   
    ]

    #  observation.state (b x n)
    ep_dict['observation.state'] = torch.stack(observation_states)
    # action (b x 2)
    ep_dict['action'] = torch.stack(actions)
    # frame_index
    ep_dict['frame_index'] = torch.arange(0, num_frames, 1)
    ep_dict['timestamp'] = torch.arange(0, num_frames, 1) / fps
    ep_dict["next.reward"] = torch.tensor(rewards)
    ep_dict["next.done"] = torch.tensor(dones)
    ep_dict["next.success"] = torch.tensor(successes)
    return ep_dict

In [29]:
def rollout_for_ep_dicts(episode_video_store, num_episodes, videos_dir):
    for _ in range(num_episodes):
        policy.reset()
        ep_dict = {}
        
        numpy_observation, info = env.reset(seed=42)
    
        frames = []
        observation_states = []
        actions = []
        rewards = []
        dones = []
        successes = []
        
        step = 0
        done = False
        while not done:
            frames.append(env.render())
            
            # Prepare observation for the policy running in Pytorch
            state = torch.from_numpy(numpy_observation["agent_pos"])
            image = torch.from_numpy(numpy_observation["pixels"])
        
            # Convert to float32 with image from channel first in [0,255]
            # to channel last in [0,1]
            state_t = state.to(torch.float32)
            image = image.to(torch.float32) / 255
            image_t = image.permute(2, 0, 1)  # c x h x w
    
            # Send data tensors from CPU to GPU
            state = state_t.to(device, non_blocking=True)
            image = image_t.to(device, non_blocking=True)
        
            # Add extra (empty) batch dimension, required to forward the policy
            state = state.unsqueeze(0)
            image = image.unsqueeze(0)
        
            # Create the policy input dictionary
            observation = {
                "observation.state": state,
                "observation.image": image,
            }
        
            # Predict the next action with respect to the current observation
            with torch.inference_mode():
                action = policy.select_action(observation)
        
            # Prepare the action for the environment
            action_t = action.squeeze(0).to("cpu")
            numpy_action = action_t.numpy()
        
            # Step through the environment and receive a new observation
            numpy_observation, reward, terminated, truncated, info = env.step(numpy_action)
            if step % 100 == 0:
                print(f"{step=} {reward=} {terminated=}")
    
            # The rollout is considered done when the success state is reach (i.e. terminated is True),
            # or the maximum number of iterations is reached (i.e. truncated is True)
            done = terminated | truncated | done
        
            # Keep track of all the rewards and frames
            observation_states.append(state_t)
            actions.append(action_t)
            rewards.append(reward)
            successes.append(terminated)
            dones.append(done)
            
            step += 1
        
        if terminated:
            print("Success!")
        else:
            print("Failure!")
        
        # Get the speed of environment (i.e. its number of frames per second).
        fps = env.metadata["render_fps"]
        
        ep_dict = build_ep_dict(observation_states=observation_states, 
                                actions=actions, 
                                rewards=rewards, 
                                dones=dones, 
                                successes=successes, 
                                frames=frames, 
                                fps=fps,
                                videos_dir=videos_dir)
        episode_video_store.add_episode(ep_dict)

    return episode_video_store

In [30]:
rollout_for_ep_dicts(episode_video_store, 5, videos_dir)

step=0 reward=0.0 terminated=False
step=100 reward=0.5850992152904797 terminated=False
step=200 reward=0.7811291648881031 terminated=False


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (680, 680) to (688, 688) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


Failure!


[swscaler @ 0x57aa940] Warning: data is not aligned! This can lead to a speed loss


Video of the evaluation is available in 'outputs/eval/episode_store_test/videos/observation.image_episode_umuMPAbY.mp4'.
step=0 reward=0.0 terminated=False
step=100 reward=0.7318638232597907 terminated=False


...

Video of the evaluation is available in 'outputs/eval/episode_store_test/videos/observation.image_episode_xpu7MYV3.mp4'.
step=0 reward=0.0 terminated=False
step=100 reward=0.5566374183822872 terminated=False
step=200 reward=0.9609870174785242 terminated=False


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (680, 680) to (688, 688) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


Failure!


[swscaler @ 0x6210940] Warning: data is not aligned! This can lead to a speed loss


Video of the evaluation is available in 'outputs/eval/episode_store_test/videos/observation.image_episode_6jTdY8q2.mp4'.


## Reload Dataset

In [31]:
episode_video_store = EpisodeVideoStore.create_from_path(output_directory, mode='r')

Connected to the exising zarr


In [32]:
dataset = episode_video_store.convert_to_lerobot_dataset('place/holder')

Generating train split: 2205 examples [00:01, 1982.43 examples/s]


In [33]:
print(f"Number of episodes: {dataset.num_episodes}")
print(f"\naverage number of frames per episode: {dataset.num_samples / dataset.num_episodes:.3f}")
print(f"frames per second used during data collection: {dataset.fps=}")
print(f"keys to access images from cameras: {dataset.camera_keys=}\n")

Number of episodes: 10

average number of frames per episode: 220.500
frames per second used during data collection: dataset.fps=10
keys to access images from cameras: dataset.camera_keys=['observation.image']



In [34]:
episode_index = 9


from_idx = dataset.episode_data_index["from"][episode_index].item()
to_idx = dataset.episode_data_index["to"][episode_index].item()

print(f"episode {episode_index} start from index {from_idx} to index {to_idx}")

frames = [dataset[idx]["observation.image"] for idx in range(from_idx, to_idx)]

# Video frames are now float32 in range [0,1] channel first (c,h,w) to follow pytorch convention. To visualize
# them, we convert to uint8 in range [0,255]
frames = [(frame * 255).type(torch.uint8) for frame in frames]
# and to channel last (h,w,c).
frames = [frame.permute((1, 2, 0)).numpy() for frame in frames]

Path("outputs/examples/1_load_lerobot_dataset").mkdir(parents=True, exist_ok=True)
video_path = f"outputs/examples/1_load_lerobot_dataset/episode_{episode_index}.mp4"
imageio.mimsave(video_path, frames, fps=dataset.fps)

# 비디오 표시
Video(video_path, embed=True, width=640, height=360)

episode 9 start from index 1905 to index 2205
